<a href="https://colab.research.google.com/github/mittushaji25/crypto-xrp-analysis/blob/main/01_xrpusdt_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests, zipfile, io
import pandas as pd
from datetime import datetime

In [9]:
years = [2023, 2024, 2025]
months = [f"{i:02d}" for i in range(1, 13)]

all_data = pd.DataFrame()

for year in years:
    for month in months:
        # Skip future months in 2025
        if year == 2025 and int(month) > datetime.now().month:
            continue

        url = f"https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1d/XRPUSDT-1d-{year}-{month}.zip"
        try:
            response = requests.get(url)
            response.raise_for_status()

            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                z.extractall("xrpusdt_data")
                csv_name = z.namelist()[0]

            df = pd.read_csv(f"xrpusdt_data/{csv_name}", header=None)
            df.columns = [
                "Open Time", "Open", "High", "Low", "Close", "Volume",
                "Close Time", "Quote Asset Volume", "Number of Trades",
                "Taker Buy Base Volume", "Taker Buy Quote Volume", "Ignore"
            ]

            all_data = pd.concat([all_data, df], ignore_index=True)
        except Exception as e:
            print(f"❌ Failed for {year}-{month}: {e}")


❌ Failed for 2025-07: 404 Client Error: Not Found for url: https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1d/XRPUSDT-1d-2025-07.zip


In [10]:
all_data.to_csv("xrpusdt_combined.csv", index=False)